## Final Project
## Weather Classification from Videos/Images using Convolution Neural Networks
## Bhuwan Sapkota and Samrid KC

### Importing the necessary libraries

In [1]:
import cv2                                # for capturing videos
import glob
import pydot
import math                               # for mathematical operations
import matplotlib.pyplot as plt           # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image     # for preprocessing the images
import numpy as np                        # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize      # for resizing images

/Users/Bhuwan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Reading the tsv files from behavioral_data and extracting the weather tags

In [ ]:
import csv 
labelArray = []

videoLabel = None
for videoFile in glob.glob('behavioral_data/*.tsv'):
    with open(videoFile) as tsvin:
        lis = [line.split() for line in tsvin]
        videoLabel = lis[16][1]
        labelArray.append([videoFile[videoFile.index('/')+1:-4],videoLabel])
        

### Label Array into Label Dictionary

In [ ]:
#labelArray = labelArray[1:]
labelDict = dict(labelArray)

In [ ]:
labelDict

In [ ]:
labelDict["video_0223"]

In [ ]:
len(labelDict)

### Extracting frames from every video and making the tags ready for them in array

In [ ]:
frameName = []
frameLabel = []

for videoFile in glob.glob('trial/*.mp4'):
           
    count = 0
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            filename = videoFile[:-4] +"_%d.jpg" % count;
            frameName.append(filename[filename.index('/')+1:])   
            frameLabel.append(labelDict[videoFile[videoFile.index('/')+1:-4]])
            count+=1
            cv2.imwrite(filename, frame)         
    cap.release()

In [ ]:
frameName

In [ ]:
len(frameLabel)

### Tagging each frame of each video clip

In [ ]:
frameName = np.asarray(frameName)
frameLabel = np.asarray(frameLabel)

frameName = frameName.reshape((-1, 1))
frameLabel = frameLabel.reshape((-1, 1))

In [ ]:
df = pd.DataFrame((np.hstack((np.asarray(frameName), np.asarray(frameLabel)))), columns = ['imageID', 'Class'])

In [ ]:
df

In [ ]:
df.to_csv('Label.csv', sep=',',index=False)  # writing to csv

In [ ]:
img = plt.imread('trial/video_0029_0.jpg')   # reading image using its name
plt.imshow(img)

### Reading all the frames and storing them in the array

In [ ]:
data = pd.read_csv('Label.csv', keep_default_na = False)

X = []
for frame in data.imageID:
    img = plt.imread("trial/" + frame)
    X.append(img)
X = np.array(X)

In [ ]:
plt.imshow(X[0])

### Reading and storing the labels

In [ ]:
y = data.Class

In [ ]:
y

In [ ]:
np.unique(y)

In [ ]:
y[45]

### Label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoderY = LabelEncoder()
y = labelEncoderY.fit_transform(y)

In [ ]:
y

In [ ]:
y = np.asarray(y)
y = y.reshape((-1, 1))

In [ ]:
y

### One hot encoding

In [ ]:
oneHotEncoderY = OneHotEncoder(categorical_features=[0]) #feature index = 0
y = oneHotEncoderY.fit_transform(y).toarray()

In [ ]:
y

### Resizing the images

In [ ]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(128,228)).astype(int)      # reshaping to 128*128*3
    image.append(a)
X = np.array(image)

In [ ]:
plt.imshow(X[0])

### Splitting the data into training and validation

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=101101219)    # preparing the validation set

In [ ]:
X_train.shape

In [ ]:
plt.imshow(X_train[0])

In [ ]:
#X_train = X_train.reshape(X_train.shape[0],3,128,228)

In [ ]:
X_train.shape 

In [ ]:
#X_valid = X_valid.reshape(X_valid.shape[0],3,128,228)

In [ ]:
X_valid.shape

In [ ]:
y_train.shape

In [ ]:
y_valid.shape

In [ ]:
num_of_classes = y_valid.shape[1]

In [ ]:
num_of_classes

### Zero centering the data

In [ ]:
X_train = X_train/X_train.max()
X_valid = X_valid/X_valid.max()

In [ ]:
plt.imshow(X_train[0])

# CNN Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers import Flatten


# Let's say you're working with 128x128 pixel RGB images (that's 128x128 pixels with 3 color channels).
# When you put such an image into a numpy array you can either store it with a shape of (128, 128, 3) 
# or with a shape of (3, 128, 128).
# The dimension ordering specifies if the color channel comes first (as with theano / "th") or if it 
# comes last (as with tensorflow / "tf").
from keras import backend as K
K.set_image_dim_ordering('tf')


def largerCNN():
    model = Sequential()
    
    #First convolution layer
    model.add(
        Conv2D(filters=48,        #number of filters
               kernel_size=(3,3),     #size of each filter
               input_shape=(128,228,3),
               activation='relu',
               strides = (1,1),
               name = "ConvolutionLayer1"
              )
    )
    
    #Pooling layer
    model.add(
        MaxPooling2D(pool_size=(2,2),
        name = "MaxPoolingLayer1")
    )
    
    #second convolution layer
    model.add(
        Conv2D(filters=96,        #number of filters
               kernel_size=(3,3),     #size of each filter
               activation='relu',
               strides = (1,1),
               name = "ConvolutionLayer2"
              )
    )
    
    #Pooling layer
    model.add(
        MaxPooling2D(pool_size=(2,2),
        name = "MaxPoolingLayer2")
    )
    
    #Flattening layer
    model.add(
        Flatten(name = "FlatteningLayer")      
    )
    
    #Fully connected layer
    model.add(
        Dense(
            units=100,
            activation='relu',
            name = "FullyConnectedLayer"
        )
    )
    
    #Output layer
    model.add(
        Dense(
            units=num_of_classes,
            activation='softmax',
            name = "OutputLayer"
        )
    )
    
    #compile model
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    #return the model
    return model

In [ ]:
# build the model
model = largerCNN()
# Print the model
print(model.summary())

In [ ]:
#Visualize the model
#Pre-req: conda install pydot graphviz
#And/or:  sudo apt install graphviz libgraphviz-dev

from keras.utils.vis_utils import plot_model
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
plot_model(model, to_file='model_plot_CNN.png', show_shapes=True, show_layer_names=True)
#Best way to display the image
from IPython.display import Image,SVG
Image(filename='model_plot_CNN.png') 
## SVG(keras.utils.vis_utils.model_to_dot(model).create(prog=’dot’, format=’svg’))

In [ ]:
# current timestamp as the log filename 
import time
t = time.localtime(time.time())
timeStamp = str(t.tm_year) + '-' + str(t.tm_mon) + '-' + str(t.tm_mday) + '--' + str(t.tm_hour) + '-'+str(t.tm_min) + '-'+str(t.tm_sec)
timeStamp

In [ ]:
## Create a TensorBoard instance with the path to the logs directory
from keras.callbacks import TensorBoard
tBoard = TensorBoard(log_dir='logs/{}'.format(timeStamp))

### Fit the model, and record history of training results

In [ ]:
# define the params
num_epochs = 10
batch_size = 35

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_valid,y_valid),
    epochs=num_epochs,
    batch_size=batch_size,
    verbose=2,
    callbacks=[tBoard]
)

In [ ]:
# Save entire model to a HDF5 file
model.save('my_model.h5')

In [3]:
import keras
# Recreate the exact same model, including weights and optimizer.
model = keras.models.load_model('my_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ConvolutionLayer1 (Conv2D)   (None, 126, 226, 48)      1344      
_________________________________________________________________
MaxPoolingLayer1 (MaxPooling (None, 63, 113, 48)       0         
_________________________________________________________________
ConvolutionLayer2 (Conv2D)   (None, 61, 111, 96)       41568     
_________________________________________________________________
MaxPoolingLayer2 (MaxPooling (None, 30, 55, 96)        0         
_________________________________________________________________
FlatteningLayer (Flatten)    (None, 158400)            0         
_________________________________________________________________
FullyConnectedLayer (Dense)  (None, 100)               15840100  
_________________________________________________________________
OutputLayer (Dense)          (None, 4)                 404       
Total para

In [4]:
# Final evaluation of the model
scores = model.evaluate(X_valid,y_valid,verbose=0)
print('Baseline error: %.2f' % (1-scores[1]))

NameError: name 'X_valid' is not defined

In [ ]:
print("Accuracy: %.2f" % scores[1])

In [ ]:
from MiscFunctions import MiscFunctions as mf
# Print/plot the training history
mf.plot_history(history)

In [ ]:
yPred = model.predict_classes(X_valid)
yPred_probabilities = model.predict(X_valid)

In [ ]:
yPred

In [ ]:
yPred_probabilities

In [ ]:
y_valid

In [ ]:
y_valid_original=np.argmax(y_valid,axis=1)

In [ ]:
y_valid_original

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print("Classification report \n=======================")
print(classification_report(y_true=y_valid_original, y_pred=yPred))
print("Confusion matrix \n=======================")
print(confusion_matrix(y_true=y_valid_original, y_pred=yPred))

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
def plot_confusion_matrix(cm, classes,
                              normalize=False,
                              title='Confusion matrix',
                              cmap=plt.cm.Blues):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix\n============================")
        else:
            print('Confusion matrix, without normalization\n============================')

        print(cm)
        print("\n")

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.tight_layout()

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_true=y_valid_original, y_pred=yPred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=np.unique(y_train),
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=np.unique(y_train), normalize=True,
                      title='Normalized confusion matrix')

plt.show()

### Now time for prediction
#### Please change the video name that needs a prediction

In [ ]:
videoFile = 'predict/video_0057.mp4'
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
count = 0
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):    # we capturing one picture every 15 frame
        filename = videoFile[:-4] +"_%d.jpg" % count;
        count+=1
        cv2.imwrite(filename, frame)

cap.release()
        

In [ ]:
predictImage = []
for frame in glob.glob('predict/*.jpg'):
    temp = plt.imread(frame)
    predictImage.append(temp)

In [ ]:
predictImage = np.asarray(predictImage)

In [ ]:
predictImage.shape

In [ ]:
plt.imshow(predictImage[0])

In [ ]:
predictedImage = []
for i in range(0,predictImage.shape[0]):
    a = resize(predictImage[i], preserve_range=True, output_shape=(128,228)).astype(int)
    predictedImage.append(a)
predictedImage = np.asarray(predictedImage)

In [ ]:
predictedImage.shape

In [ ]:
plt.imshow(predictedImage[0])

In [ ]:
#predictedImage = predictedImage.reshape(predictedImage.shape[0],3,128,228)

In [ ]:
predictions = model.predict_classes(predictedImage)

In [ ]:
predictions

In [ ]:
predictions = labelEncoderY.inverse_transform(predictions)

### Following snippet will predict the weather

In [ ]:
(values,counts) = np.unique(predictions, return_counts=True)
ind=np.argmax(counts)
print (values[ind])  # prints the most frequent element